In [ ]:
from flask import Flask, jsonify, request
from werkzeug.utils import secure_filename
from flask import render_template
from flask import make_response
from FruitsNeuralNetwork import FruitsNeuralNetwork
import torch
from time import time
from PIL import Image
import json
import numpy as np
from torchvision import transforms
import torch.nn.functional as F
from DbConnection import DbConnection

# Consultar nombre del modelo
json_params= open('params.json')
params_dict = json.load(json_params)
model_name = params_dict["model_name"]

# Cargamos el modelo
# model = FruitsNeuralNetwork()
# model.load_state_dict(torch.load(model_name))
# model.eval()
model = torch.load("fruits_model3.pth")
model.eval()
print("Definir conexión a la BD")
connection= DbConnection()
print("Realizar Consulta de imágenes")
labels_dataframe= connection.get_labels()
labels_list= labels_dataframe.iloc[:, 1]

from os import path, walk

# definimos la app
app = Flask(__name__, template_folder='web',
            static_url_path='/static',
            static_folder='web/static')
app.config['TEMPLATES_AUTO_RELOAD'] = True


@app.route("/")
def hello():
    version=int(time());
    return render_template("index.html",
                            version=version)

@app.route('/analize', methods = ['POST', 'GET'])
def upload_file():
    # Validar que se esté subiendo algo
    if request.method == 'POST':
          # Recorremos los archivos
          for i in request.files:
                # Abrimos la imagen y la formateamos al tamaño adecuado
                img = Image.open(request.files[i])
                img= img.resize((100, 100))
                to_tensor = transforms.ToTensor()
                img_tensor = to_tensor(img)
                pred_list=list(range(16))
                for i in pred_list:
                      pred_list[i] = img_tensor.numpy()
                img_tensor = torch.tensor(pred_list)
                result = get_result(model=model,images= img_tensor.cuda(), classes= labels_list)
                print(result)
                return result
def images_to_probs(model, images):
      output = model(images)
      _, preds_tensor = torch.max(output, 1)
      if torch.cuda.is_available():
            preds = np.squeeze(preds_tensor.cpu().data.numpy())
      else:
            preds = np.squeeze(preds_tensor.numpy())
      return preds, [np.around (F.softmax(el, dim=0)[i].item(),2) for i, el in zip(preds, output)]

def get_result(model, images, classes):
      preds, probs = images_to_probs(model, images)
      idx = 0
      result = {
            "clase":classes[preds[idx]],
            "probabilidad": str(probs[idx] * 100.0)
      }
      return result
if __name__ == "__main__":
  app.run()